In [ ]:
# 1. Instalar dependências
!pip install -q ultralytics roboflow

In [ ]:
# 2. Verificar GPU
import torch
print(f"GPU disponível: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# 3. Baixar dataset do Roboflow
# IMPORTANTE: Crie conta em https://roboflow.com e obtenha sua API key

from roboflow import Roboflow

# Substitua pela sua API key do Roboflow
API_KEY = "SUA_API_KEY"  # @param {type:"string"}

rf = Roboflow(api_key=API_KEY)

# Dataset com 11.763 imagens de buracos
project = rf.workspace("project-saocp").project("pothole-fy5oo")
dataset = project.version(1).download("yolov8")

print(f"\n✅ Dataset baixado em: {dataset.location}")

In [ ]:
# 4. Treinar modelo YOLO11
from ultralytics import YOLO

# Carregar modelo YOLO11 nano (leve para mobile)
model = YOLO("yolo11n.pt")

# Treinar
results = model.train(
    data=f"{dataset.location}/data.yaml",
    epochs=50,              # Ajuste conforme necessário
    imgsz=640,
    batch=16,
    device=0,
    patience=15,
    save=True,
    project="pothole_model",
    name="yolo11n",
    augment=True,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=15,
    translate=0.1,
    scale=0.5,
    fliplr=0.5,
    mosaic=1.0,
)

In [ ]:
# 5. Validar modelo
metrics = model.val()

print("\n" + "="*50)
print("📊 RESULTADOS DA VALIDAÇÃO")
print("="*50)
print(f"mAP50:     {metrics.box.map50:.1%}")
print(f"mAP50-95:  {metrics.box.map:.1%}")
print(f"Precisão:  {metrics.box.p[0]:.1%}")
print(f"Recall:    {metrics.box.r[0]:.1%}")
print("="*50)

In [ ]:
# 6. Exportar para ONNX (compatível com Android/OpenCV)
best_model = YOLO("pothole_model/yolo11n/weights/best.pt")

best_model.export(
    format="onnx",
    imgsz=320,          # Tamanho menor para mobile
    simplify=True,
    opset=12,
    dynamic=False,
)

print("\n✅ Modelo exportado para ONNX!")

In [ ]:
# 7. Baixar arquivos
from google.colab import files
import shutil

# Copiar modelo ONNX
shutil.copy("pothole_model/yolo11n/weights/best.onnx", "pothole_detector.onnx")

# Baixar
files.download("pothole_detector.onnx")

print("\n📱 Arquivo baixado! Coloque na pasta do projeto Android.")